<a href="https://colab.research.google.com/github/NekoMonci12/Machine-Learning-Brazilian-E-Commerce/blob/main/ML_Muhammad_Tamir.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Proyek Analisis Data: Brazilian E-Commerce Public
- **Nama:** Muhammad Tamir

## Menentukan Pertanyaan Bisnis

- Negara bagian (State) mana di Brazil yang memiliki konsentrasi pelanggan tertinggi, dan bagaimana distribusi total pendapatan (revenue) di antara negara bagian tersebut?
- Apakah terdapat korelasi antara lama waktu pengiriman (delivery time) dengan skor ulasan (review score) yang diberikan oleh pelanggan?

## Import Semua Packages/Library yang Digunakan

In [22]:
import os
import re
import unicodedata
import kagglehub
from kagglehub import KaggleDatasetAdapter
import pandas as pd

## Data Wrangling

### Gathering Data

#### Setup Kaggle

In [23]:
dataset_path = kagglehub.dataset_download("olistbr/brazilian-ecommerce")

files = [
    f for f in os.listdir(dataset_path)
    if f.endswith(".csv")
]

data = {}

for file_path in files:
    key = file_path.removesuffix(".csv")

    df = kagglehub.dataset_load(
        KaggleDatasetAdapter.PANDAS,
        "olistbr/brazilian-ecommerce",
        file_path,
    )

    data[key] = df
    print(f"Loaded {key} dataset")

Using Colab cache for faster access to the 'brazilian-ecommerce' dataset.
Using Colab cache for faster access to the 'brazilian-ecommerce' dataset.
Loaded olist_customers_dataset dataset
Using Colab cache for faster access to the 'brazilian-ecommerce' dataset.
Loaded olist_sellers_dataset dataset
Using Colab cache for faster access to the 'brazilian-ecommerce' dataset.
Loaded olist_order_reviews_dataset dataset
Using Colab cache for faster access to the 'brazilian-ecommerce' dataset.
Loaded olist_order_items_dataset dataset
Using Colab cache for faster access to the 'brazilian-ecommerce' dataset.
Loaded olist_products_dataset dataset
Using Colab cache for faster access to the 'brazilian-ecommerce' dataset.
Loaded olist_geolocation_dataset dataset
Using Colab cache for faster access to the 'brazilian-ecommerce' dataset.
Loaded product_category_name_translation dataset
Using Colab cache for faster access to the 'brazilian-ecommerce' dataset.
Loaded olist_orders_dataset dataset
Using Cola

**Insight:**
- xxx
- xxx

#### Setup City & State Mapping

In [24]:
state_mapping = {
    "acre": "AC",
    "alagoas": "AL",
    "amapa": "AP",
    "amazonas": "AM",
    "bahia": "BA",
    "ceara": "CE",
    "distrito federal":"DF",
    "espirito santo": "ES",
    "goias": "GO",
    "maranhao": "MA",
    "mato grosso": "MT",
    "mato grosso do sul": "MS",
    "minas gerais": "MG",
    "para": "PA",
    "paraiba": "PB",
    "parana": "PR",
    "pernambuco": "PE",
    "piaui": "PI",
    "rio de janeiro": "RJ",
    "rio grande do norte": "RN",
    "rio grande do sul": "RS",
    "rondonia": "RO", "roraima": "RR",
    "santa catarina": "SC",
    "sao paulo": "SP", "sergipe": "SE",
    "tocantins": "TO"
}

### Assessing Data

#### Dataset Declaration

In [25]:
customers = data["olist_customers_dataset"]
geolocation = data["olist_geolocation_dataset"]
order_item = data["olist_order_items_dataset"]
order_payment = data["olist_order_payments_dataset"]
order_review = data["olist_order_reviews_dataset"]
orders = data["olist_orders_dataset"]
products = data["olist_products_dataset"]
sellers = data["olist_sellers_dataset"]
product_category = data["product_category_name_translation"]

#### Dataset Function

##### Product Function

In [26]:
def get_product_category(product_id, translate=False):
    category_series = products[products['product_id'] == product_id]['product_category_name']
    if not category_series.empty:
        category = category_series.values[0]
        if translate and 'product_category_name_translation' in data:
            translation = product_category[product_category['product_category_name'] == category]['product_category_name_english']
            if not translation.empty:
                return translation.values[0]

        return category
    else:
        return "Product ID tidak ditemukan"

#### Assessing Data Result

**Insight:**
- xxx
- xxx

### Cleaning Data

#### Functions

##### Cleaning Utils

In [27]:
def normalize_unicode(text):
    if isinstance(text, str):
        return unicodedata.normalize("NFD", text)
    return text

def clean_brazilian_cities(text):
    if isinstance(text, str):
        text = text.replace('£', 'a')
        text = normalize_unicode(text)
        text = text.encode('ascii', 'ignore').decode("utf-8")
        text = re.sub(r'[^a-zA-Z\s]', '', text)

        return text.lower().strip()
    return text

def remove_duplicates_data(df, subset_columns):
    duplicate_count = df.duplicated(subset=subset_columns).sum()
    df_cleaned = df.drop_duplicates(subset=subset_columns, keep='first').copy()

    print(f"--- Laporan Pembersihan Duplikasi ---")
    print(f"Total baris awal      : {len(df)}")
    print(f"Jumlah duplikat literal: {duplicate_count}")
    print(f"Total baris sekarang  : {len(df_cleaned)}")
    print(f"-------------------------------------")

    return df_cleaned

def remove_mismatched_data(df_target, col_target, df_ref, col_ref):
    valid_values = df_ref[col_ref].unique()
    initial_count = len(df_target)
    df_cleaned = df_target[df_target[col_target].isin(valid_values)].copy()
    removed_count = initial_count - len(df_cleaned)

    print(f"--- Laporan Pembersihan Integritas ---")
    print(f"Dataset Target       : {initial_count} baris")
    print(f"Data tidak ditemukan : {removed_count} baris dihapus")
    print(f"Dataset Akhir        : {len(df_cleaned)} baris")
    print(f"---------------------------------------")

    return df_cleaned

def remove_null_rows(df, subset_columns):
    initial_count = len(df)
    df_cleaned = df.dropna(subset=subset_columns, how='any').copy()
    removed_count = initial_count - len(df_cleaned)

    print(f"--- Laporan Penanganan Missing Value ---")
    print(f"Dataset             : {initial_count} baris")
    print(f"Baris dengan null    : {removed_count} baris dihapus")
    print(f"Dataset Bersih      : {len(df_cleaned)} baris")
    print(f"----------------------------------------")

    return df_cleaned

##### City & State Validation

In [28]:
geo_reference = customers[['customer_city', 'customer_state']].drop_duplicates()
def check_location(city=None, state=None):
    if city: city = clean_brazilian_cities(city)
    if state: state = state.upper()

    if city and state:
        exists = not geo_reference[(geo_reference['customer_city'] == city) &
                                   (geo_reference['customer_state'] == state)].empty
        return exists

    elif city:
        states = geo_reference[geo_reference['customer_city'] == city]['customer_state'].unique()
        return f"{list(states)}" if len(states) > 0 else "Kota tidak ditemukan."

    elif state:
        cities = geo_reference[geo_reference['customer_state'] == state]['customer_city'].unique()
        return f"{list(cities[:5])}" if len(cities) > 0 else "State tidak ditemukan."

##### Date & Time Utils

In [29]:
def convert_to_datetime(df, column_name):
    df[column_name] = pd.to_datetime(df[column_name], errors='coerce')

    return df

#### Alter Format

##### Date & Time String To Datetime

In [30]:
orders = convert_to_datetime(orders, 'order_purchase_timestamp')
orders = convert_to_datetime(orders, 'order_approved_at')
orders = convert_to_datetime(orders, 'order_delivered_carrier_date')
orders = convert_to_datetime(orders, 'order_delivered_customer_date')
orders = convert_to_datetime(orders, 'order_estimated_delivery_date')

#### Cleaning Process

##### Customers Dataset

In [31]:
remove_duplicates_data(customers, ['customer_unique_id'])

--- Laporan Pembersihan Duplikasi ---
Total baris awal      : 99441
Jumlah duplikat literal: 3345
Total baris sekarang  : 96096
-------------------------------------


,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,SP
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP
...,...,...,...,...,...
99436,17ddf5dd5d51696bb3d7c6291687be6f,1a29b476fee25c95fbafc67c5ac95cf8,3937,sao paulo,SP
99437,e7b71a9017aa05c9a7fd292d714858e8,d52a67c98be1cf6a5c84435bd38d095d,6764,taboao da serra,SP
99438,5e28dfe12db7fb50a4b2f691faecea5e,e9f50caf99f032f0bf3c55141f019d99,60115,fortaleza,CE
99439,56b18e2166679b8a959d72dd06da27f9,73c2643a0a458b49f58cea58833b192e,92120,canoas,RS


##### Product Dataset

In [32]:
remove_null_rows(products, ["product_category_name"])

--- Laporan Penanganan Missing Value ---
Dataset             : 32951 baris
Baris dengan null    : 610 baris dihapus
Dataset Bersih      : 32341 baris
----------------------------------------


,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumaria,40.0,287.0,1.0,225.0,16.0,10.0,14.0
1,3aa071139cb16b67ca9e5dea641aaa2f,artes,44.0,276.0,1.0,1000.0,30.0,18.0,20.0
2,96bd76ec8810374ed1b65e291975717f,esporte_lazer,46.0,250.0,1.0,154.0,18.0,9.0,15.0
3,cef67bcfe19066a932b7673e239eb23d,bebes,27.0,261.0,1.0,371.0,26.0,4.0,26.0
4,9dc1a7de274444849c219cff195d0b71,utilidades_domesticas,37.0,402.0,4.0,625.0,20.0,17.0,13.0
...,...,...,...,...,...,...,...,...,...
32946,a0b7d5a992ccda646f2d34e418fff5a0,moveis_decoracao,45.0,67.0,2.0,12300.0,40.0,40.0,40.0
32947,bf4538d88321d0fd4412a93c974510e6,construcao_ferramentas_iluminacao,41.0,971.0,1.0,1700.0,16.0,19.0,16.0
32948,9a7c6041fa9592d9d9ef6cfe62a71f8c,cama_mesa_banho,50.0,799.0,1.0,1400.0,27.0,7.0,27.0
32949,83808703fc0706a22e264b9d75f04a2e,informatica_acessorios,60.0,156.0,2.0,700.0,31.0,13.0,20.0


##### Order Dataset

In [33]:
remove_mismatched_data(order_item, 'order_id', orders, 'order_id')

--- Laporan Pembersihan Integritas ---
Dataset Target       : 112650 baris
Data tidak ditemukan : 0 baris dihapus
Dataset Akhir        : 112650 baris
---------------------------------------


,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14
...,...,...,...,...,...,...,...
112645,fffc94f6ce00a00581880bf54a75a037,1,4aa6014eceb682077f9dc4bffebc05b0,b8bc237ba3788b23da09c0f1f3a3288c,2018-05-02 04:11:01,299.99,43.41
112646,fffcd46ef2263f404302a634eb57f7eb,1,32e07fd915822b0765e448c4dd74c828,f3c38ab652836d21de61fb8314b69182,2018-07-20 04:31:48,350.00,36.53
112647,fffce4705a9662cd70adb13d4a31832d,1,72a30483855e2eafc67aee5dc2560482,c3cfdc648177fdbbbb35635a37472c53,2017-10-30 17:14:25,99.90,16.95
112648,fffe18544ffabc95dfada21779c9644f,1,9c422a519119dcad7575db5af1ba540e,2b3e4a2a3ea8e01938cabda2a3e5cc79,2017-08-21 00:04:32,55.99,8.72


In [34]:
remove_mismatched_data(order_payment, 'order_id', orders, 'order_id')

--- Laporan Pembersihan Integritas ---
Dataset Target       : 103886 baris
Data tidak ditemukan : 0 baris dihapus
Dataset Akhir        : 103886 baris
---------------------------------------


,order_id,payment_sequential,payment_type,payment_installments,payment_value
0,b81ef226f3fe1789b1e8b2acac839d17,1,credit_card,8,99.33
1,a9810da82917af2d9aefd1278f1dcfa0,1,credit_card,1,24.39
2,25e8ea4e93396b6fa0d3dd708e76c1bd,1,credit_card,1,65.71
3,ba78997921bbcdc1373bb41e913ab953,1,credit_card,8,107.78
4,42fdf880ba16b47b59251dd489d4441a,1,credit_card,2,128.45
...,...,...,...,...,...
103881,0406037ad97740d563a178ecc7a2075c,1,boleto,1,363.31
103882,7b905861d7c825891d6347454ea7863f,1,credit_card,2,96.80
103883,32609bbb3dd69b3c066a6860554a77bf,1,credit_card,1,47.77
103884,b8b61059626efa996a60be9bb9320e10,1,credit_card,5,369.54


In [35]:
remove_mismatched_data(order_review, 'order_id', orders, 'order_id')

--- Laporan Pembersihan Integritas ---
Dataset Target       : 99224 baris
Data tidak ditemukan : 0 baris dihapus
Dataset Akhir        : 99224 baris
---------------------------------------


,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
0,7bc2406110b926393aa56f80a40eba40,73fc7af87114b39712e6da79b0a377eb,4,NaN,NaN,2018-01-18 00:00:00,2018-01-18 21:46:59
1,80e641a11e56f04c1ad469d5645fdfde,a548910a1c6147796b98fdf73dbeba33,5,NaN,NaN,2018-03-10 00:00:00,2018-03-11 03:05:13
2,228ce5500dc1d8e020d8d1322874b6f0,f9e4b658b201a9f2ecdecbb34bed034b,5,NaN,NaN,2018-02-17 00:00:00,2018-02-18 14:36:24
3,e64fb393e7b32834bb789ff8bb30750e,658677c97b385a9be170737859d3511b,5,NaN,Recebi bem antes do prazo estipulado.,2017-04-21 00:00:00,2017-04-21 22:02:06
4,f7c4243c7fe1938f181bec41a392bdeb,8e6bfb81e283fa7e4f11123a3fb894f1,5,NaN,Parabéns lojas lannister adorei comprar pela I...,2018-03-01 00:00:00,2018-03-02 10:26:53
...,...,...,...,...,...,...,...
99219,574ed12dd733e5fa530cfd4bbf39d7c9,2a8c23fee101d4d5662fa670396eb8da,5,NaN,NaN,2018-07-07 00:00:00,2018-07-14 17:18:30
99220,f3897127253a9592a73be9bdfdf4ed7a,22ec9f0669f784db00fa86d035cf8602,5,NaN,NaN,2017-12-09 00:00:00,2017-12-11 20:06:42
99221,b3de70c89b1510c4cd3d0649fd302472,55d4004744368f5571d1f590031933e4,5,NaN,"Excelente mochila, entrega super rápida. Super...",2018-03-22 00:00:00,2018-03-23 09:10:43
99222,1adeb9d84d72fe4e337617733eb85149,7725825d039fc1f0ceb7635e3f7d9206,4,NaN,NaN,2018-07-01 00:00:00,2018-07-02 12:59:13


In [36]:
remove_mismatched_data(order_item, 'product_id', products, 'product_id')

--- Laporan Pembersihan Integritas ---
Dataset Target       : 112650 baris
Data tidak ditemukan : 0 baris dihapus
Dataset Akhir        : 112650 baris
---------------------------------------


,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14
...,...,...,...,...,...,...,...
112645,fffc94f6ce00a00581880bf54a75a037,1,4aa6014eceb682077f9dc4bffebc05b0,b8bc237ba3788b23da09c0f1f3a3288c,2018-05-02 04:11:01,299.99,43.41
112646,fffcd46ef2263f404302a634eb57f7eb,1,32e07fd915822b0765e448c4dd74c828,f3c38ab652836d21de61fb8314b69182,2018-07-20 04:31:48,350.00,36.53
112647,fffce4705a9662cd70adb13d4a31832d,1,72a30483855e2eafc67aee5dc2560482,c3cfdc648177fdbbbb35635a37472c53,2017-10-30 17:14:25,99.90,16.95
112648,fffe18544ffabc95dfada21779c9644f,1,9c422a519119dcad7575db5af1ba540e,2b3e4a2a3ea8e01938cabda2a3e5cc79,2017-08-21 00:04:32,55.99,8.72


##### Cleaning Results


**Insight:**
- xxx
- xxx

## Exploratory Data Analysis (EDA)

### Explore ...

**Insight:**
- xxx
- xxx

## Visualization & Explanatory Analysis

### Pertanyaan 1:

### Pertanyaan 2:

**Insight:**
- xxx
- xxx

## Analisis Lanjutan (Opsional)

## Conclusion

- Conclution pertanyaan 1
- Conclution pertanyaan 2